# 도구 (Tools)

도구(Tool)는 에이전트, 체인 또는 LLM이 외부 세계와 상호작용하기 위한 인터페이스입니다.

LangChain 에서 기본 제공하는 도구를 사용하여 쉽게 도구를 활용할 수 있으며, 사용자 정의 도구(Custom Tool) 를 쉽게 구축하는 것도 가능합니다.

In [3]:
from dotenv import load_dotenv
from langchain_teddynote import logging
import warnings


load_dotenv()
logging.langsmith("CH15-Tools")
warnings.filterwarnings("ignore")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH15-Tools


## 빌트인 도구(built-in tools)

체인에서 제공하는 사전에 정의된 도구(tool) 와 툴킷(toolkit) 을 사용할 수 있습니다.

tool 은 단일 도구를 의미하며, toolkit 은 여러 도구를 묶어서 하나의 도구로 사용할 수 있습니다.

### Python REPL 도구

**설명**
- Python 셸 환경을 제공합니다.
- 유효한 Python 명령어를 입력으로 받아 실행합니다.
- 결과를 보려면 print(...) 함수를 사용해야 합니다.

**주요 특징**
- sanitize_input: 입력을 정제하는 옵션 (기본값: True)
- python_repl: PythonREPL 인스턴스 (기본값: 전역 범위에서 실행)
- 사용 방법

**PythonREPLTool 인스턴스 생성**
- run 또는 arun, invoke 메서드를 사용하여 Python 코드 실행

**입력 정제**
- 입력 문자열에서 불필요한 공백, 백틱, 'python' 키워드 등을 제거합니다.

In [5]:
from langchain_experimental.tools import PythonREPLTool

python_tool = PythonREPLTool()

# 파이썬 코드를 실행하고 결과를 반환합니다.
print(python_tool.invoke("print(100 + 200)"))

Python REPL can execute arbitrary code. Use with caution.


300



In [6]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableLambda


# 파이썬 코드를 실행하고 중간 과정을 출력하고 도구 실행 결과를 반환하는 함수
def print_and_execute(code, debug=True):
    if debug:
        print("CODE:")
        print(code)
    return python_tool.invoke(code)


# 파이썬 코드를 작성하도록 요청하는 프롬프트
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are Raymond Hetting, an expert python programmer, well versed in meta-programming and elegant, concise and short but well documented code. You follow the PEP8 style guide. "
            "Return only the code, no intro, no explanation, no chatty, no markdown, no code block, no nothing. Just the code.",
        ),
        ("human", "{input}"),
    ]
)
# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트와 LLM 모델을 사용하여 체인 생성
chain = prompt | llm | StrOutputParser() | RunnableLambda(print_and_execute)

print(chain.invoke("로또 번호 생성기를 출력하는 코드를 작성하세요."))

CODE:
import random

def generate_lotto_numbers():
    return sorted(random.sample(range(1, 46), 6))

print(generate_lotto_numbers())
[7, 19, 22, 35, 36, 39]



### TavilySearchResults

**설명**
- Tavily 검색 API를 쿼리하고 JSON 형식의 결과를 반환합니다.
- 포괄적이고 정확하며 신뢰할 수 있는 결과에 최적화된 검색 엔진입니다.
- 현재 이벤트에 대한 질문에 답변할 때 유용합니다.

**주요 매개변수**
- max_results (int): 반환할 최대 검색 결과 수 (기본값: 5)
- search_depth (str): 검색 깊이 ("basic" 또는 "advanced")
- include_domains (List[str]): 검색 결과에 포함할 도메인 목록
- exclude_domains (List[str]): 검색 결과에서 제외할 도메인 목록
- include_answer (bool): 원본 쿼리에 대한 짧은 답변 포함 여부
- include_raw_content (bool): 각 사이트의 정제된 HTML 콘텐츠 포함 여부
- include_images (bool): 쿼리 관련 이미지 목록 포함 여부

In [8]:
from langchain_community.tools.tavily_search import TavilySearchResults

# 도구 생성
tool = TavilySearchResults(
    max_results=6,
    include_answer=True,
    include_raw_content=True,
    # include_images=True,
    # search_depth="advanced", # or "basic"
    include_domains=["github.io", "naver.com"],
    exclude_domains = ["google.com"]
)

tool.invoke({"query": "LangChain Tools 에 대해서 알려주세요"})

[{'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=37460860&vType=VERTICAL',
  'content': '랭체인 (LangChain) 은 언어 모델 기반의 애플리케이션을 개발하는 프레임워크입니다. 랭체인을 사용해 챗봇 또는 개인 비서를 만들고 , 문서 또는 구조화된 데이터에 대한 Q&A 를 요약 , 분석 , 생성하고 , 코드를 쓰거나 이해하고 , API 와 상호작용하고 , 생성형 AI 를'},
 {'url': 'https://post.naver.com/viewer/postView.naver?volumeNo=38409843&vType=VERTICAL',
  'content': 'LangChain 을 통해 개발자는 더 복잡한 사용 사례를 처리할 수 있는 애플리케이션을 손쉽게 구축할 수 있습니다. LangChain 은 오픈소스 프로젝트로서, 전 세계 개발자들이 참여하고 기여하는 커뮤니티에 의해 지속적으로 발전하고 있습니다.'},
 {'url': 'https://blog.naver.com/PostView.naver?blogId=beyond-zero&logNo=223386007783',
  'content': 'ChatGPT와 같은 언어 모델은 매우 다재다능하지만 비즈니스 현장에서 그대로 사용할 수 있는 4가지 제한사항이 있다. 이런 4가지 제한 사항을 LangChain을 사용하면 해결할 수 있다. LangChain: 사내의 높은 보안수준에 대응한 인프라인 환경에서 애플리케이션을 실행할 수 있다. LangChain: ChatGPT의 공식 서비스에 없는 임의의 사용자지정 기능을 추가할 수 있다. 또한, Chains를 사용하여 프롬프트 출력을 결합할 수 있다. 예를 들어, A와 B의 두 프롬프트 출력을 사용하여 프롬프트를 실행할 수 있다. 이 기능을 사용하면 모델이 복잡한 작업을 자동으로 처리할 수 있다. 즉, 이 기능을 사용하면 개발자가 애플리케이션의 동작을 보다 세밀하게 제어하고 관리할 수

## 사용자 정의 도구(Custom Tool)

LangChain 에서 제공하는 빌트인 도구 외에도 사용자가 직접 도구를 정의하여 사용할 수 있습니다. 이를 위해서는 langchain.tools 모듈에서 제공하는 tool 데코레이터를 사용하여 함수를 도구로 변환합니다.

### @tool 데코레이터

이 데코레이터는 함수를 도구로 변환하는 기능을 제공합니다. 다양한 옵션을 통해 도구의 동작을 커스터마이즈할 수 있습니다.

***사용 방법**

함수 위에 @tool 데코레이터 적용

필요에 따라 데코레이터 매개변수 설정

In [10]:
from langchain.tools import tool

@tool
def add_numbers(a: int, b: int) -> int:
    """Add two Numbers"""
    return a + b

@tool
def multiply_numbers(a: int, b: int) -> int:
    """Multiply two numbers"""
    return a * b

# 도구 실행
add_numbers.invoke({"a": 3, "b": 4})

# 도구 실행
multiply_numbers.invoke({"a": 3, "b": 4})

12